# side quest

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm


surface1 = tlm.surfaces.Parabola(15.0, 0.020)
surface2 = tlm.surfaces.Parabola(15.0, 0.030)
surface3 = tlm.surfaces.Sphere(12, -20)

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(18., 0),
    tlm.Gap(5),
    tlm.RefractiveSurface(surface1, n=(1.0, 1.5), anchors=("origin", "extent")),
    tlm.Gap(1),
    tlm.RefractiveSurface(surface1, n=(1.5, 1.0), scale=-1, anchors=("extent", "origin")),
    tlm.Gap(0),
    tlm.Aperture(10),
    tlm.Gap(15),
    tlm.ReflectiveSurface(surface3),
)


tlm.show(optics, mode="2D", end=10)
tlm.show(optics, mode="3D", end=10)

In [ ]:
# mode 1:  inline / chained / affecting
# surface transform = input.transforms - anchor + scale
# output transform = input.transforms - first anchor + second anchor

# mode 2: offline / free / independent
# surface transform = input.transforms + local transform - anchor + scale
# output transform = input.transforms

# how to support absolute position on chain?

# RS(X - A) + T
# surface transform(X) = CSX - A
# surface transform = anchor1 + scale + chain
# output transform = chain + anchor1 + anchor2